# Spark SQL with pySpark

Answer the following questions

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Row, Window
from pyspark.sql.types import IntegerType

In [2]:
spark = (
    SparkSession
    .builder
    .master("local")
    .config("spark.sql.autoBroadcastJoinThreshold", -1)
    .config("spark.executor.memory", "3g")
    .appName("Exercise1")
    .getOrCreate()
)

In [3]:
# Read the source tables
products_table = spark.read.parquet("./data/products_parquet")
sales_table    = spark.read.parquet("./data/sales_parquet")
sellers_table  = spark.read.parquet("./data/sellers_parquet")

In [4]:
products_table.printSchema()
sales_table.printSchema()
sellers_table.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: string (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- num_pieces_sold: string (nullable = true)
 |-- bill_raw_text: string (nullable = true)

root
 |-- seller_id: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: string (nullable = true)



# Data study

For each seller find the average performance as a % of their daily target amount

|seller_id|avg_daily_percentage|
|--------:|--------------------|
|        1|              212.22|
|        2|               72.99|
|        3|              169.68|
|        4|               35.75|
|        5|               45.66|
|        6|               52.91|
|        7|                28.1|
|        8|              102.76|
|        9|               40.81|

Find out how many sellers are not making their daily goal for every day.

only showing top 20 rows:

|seller_name|      date|sellers_daily_profit|daily_target|
|----------:|----------|--------------------|------------|
|   seller_2|2020-07-01|            518119.0|      754188|
|   seller_2|2020-07-02|            512012.0|      754188|
|   seller_2|2020-07-03|            525180.0|      754188|
|   seller_2|2020-07-04|            617840.0|      754188|
|   seller_2|2020-07-05|            508774.0|      754188|
|   seller_2|2020-07-06|            589736.0|      754188|
|   seller_2|2020-07-07|            635401.0|      754188|
|   seller_2|2020-07-08|            604797.0|      754188|
|   seller_2|2020-07-09|            503370.0|      754188|
|   seller_2|2020-07-10|            487760.0|      754188|
|   seller_4|2020-07-01|            549595.0|     1532808|
|   seller_4|2020-07-02|            512200.0|     1532808|
|   seller_4|2020-07-03|            627174.0|     1532808|
|   seller_4|2020-07-04|            519878.0|     1532808|
|   seller_4|2020-07-05|            486912.0|     1532808|
|   seller_4|2020-07-06|            563439.0|     1532808|
|   seller_4|2020-07-07|            531202.0|     1532808|
|   seller_4|2020-07-08|            584587.0|     1532808|
|   seller_4|2020-07-09|            508752.0|     1532808|
|   seller_4|2020-07-10|            596303.0|     1532808|

What sellers have the biggest diffference between the day they performed the best and the day they performed the worst?

|seller_id|min_daily_percentage|max_daily_percentage|min_max_difference|
|--------:|--------------------|--------------------|------------------|
|        7|               23.43|                31.3|              7.87|
|        4|               31.77|               40.92|              9.15|
|        6|               46.13|               57.49|             11.36|
|        9|               32.52|               46.83|             14.31|
|        5|               37.56|               52.02|             14.46|
|        2|               64.67|               84.25|             19.58|
|        8|               87.55|              114.94|             27.39|
|        1|              193.52|               227.6|             34.08|
|        3|              145.56|              197.62|             52.06|

# Pivot tables

Let's imagine that there is a "product category", based on the first number of the product id. To create this column, you can use the functions [lit](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.lit.html#) and [concat](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.concat.html), and the property [substr](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.substr.html) of a column

Find out about whether or not it is more common to sell more units during the weekend, depending on product category. Use a [pivot table](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.GroupedData.pivot.html).

TIP: what metric should you use to know whether is it more common to sell during the weekend or not?

I'm not telling you the results! But it should look like this:

|weekend|category_1|category_2|category_3|category_4|category_5|category_6|category_7|category_8|category_9|
|------:|-----|-----|-----|-----|-----|-----|-----|-----|-----|
|   true|  -  |  -  |  -  |  -  |  -  |  -  |  -  |  -  |  -  |
|  false|  -  |  -  |  -  |  -  |  -  |  -  |  -  |  -  |  -  |

Go back to the exercise of "which sellers are not making their daily goal" and study the percentage of days they make it, depending on the day of week. Can you find out any interesting information?

Again, I'm not telling you the results.

|seller_name|    1|    2|    3|    4|    5|    6|    7|
|----------:|-----|-----|-----|-----|-----|-----|-----|
|   seller_1|  -  |  -  |  -  |  -  |  -  |  -  |  -  |
|   seller_2|  -  |  -  |  -  |  -  |  -  |  -  |  -  |
|   seller_3|  -  |  -  |  -  |  -  |  -  |  -  |  -  |
|   seller_4|  -  |  -  |  -  |  -  |  -  |  -  |  -  |
|   seller_5|  -  |  -  |  -  |  -  |  -  |  -  |  -  |
|   seller_6|  -  |  -  |  -  |  -  |  -  |  -  |  -  |
|   seller_7|  -  |  -  |  -  |  -  |  -  |  -  |  -  |
|   seller_8|  -  |  -  |  -  |  -  |  -  |  -  |  -  |
|   seller_9|  -  |  -  |  -  |  -  |  -  |  -  |  -  |


# Window functions

[Window functions](https://medium.com/@uzzaman.ahmed/pyspark-window-functions-a-comprehensive-guide-dc9bdad8c7ae) allow us to query subsets of the data without the need to do things like creating another group by table and joining the two.

See [the documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/window.html) for everything you can do with window functions.

Which seller, and on which day, has made the most money compared to their previous day? Which one has made the least?

|seller_name|      date|profit_percent_of_prev_day|
|----------:|----------|--------------------------|
|   seller_5|2020-07-09|                      77.0|
|   seller_9|2020-07-03|                     143.0|

Coming back to the "category" we set before, find out, for each category, which salesman is the best, the second best and the worst at making profit from it.

Tip: you can use a pivot table if you assign a value for each seller, according to their position in the category (look at the documentation for window functions!) and then rename using the property [withColumnRenamed](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumnRenamed.html).

|  category|    best|second_best|   worst|
|---------:|--------|-----------|--------|
|category_1|seller_6|   seller_2|seller_8|
|category_2|seller_2|   seller_8|seller_3|
|category_3|seller_8|   seller_1|seller_6|
|category_4|seller_7|   seller_1|seller_4|
|category_5|seller_8|   seller_4|seller_2|
|category_6|seller_6|   seller_3|seller_5|
|category_7|seller_6|   seller_4|seller_9|
|category_8|seller_9|   seller_4|seller_3|
|category_9|seller_2|   seller_5|seller_6|